In [1]:
import pandas as pd
import requests
import json
import os

# Authenticate with Bayern Cloud

In [2]:
# Load Bayern Cloud API key from environment variables
BAYERN_CLOUD_API_KEY = os.getenv('BAYERN_CLOUD_API_KEY')

# Load occupancy data

## Real-time occupancy data

In [3]:
def load_and_save_occupancy_data_for_location(
    location_slug: str,
    save_file_path: str = 'outputs'
):
    API_endpoint = f'https://data.bayerncloud.digital/api/v4/endpoints/list_occupancy/{location_slug}'

    request_params = {
        'token': BAYERN_CLOUD_API_KEY
    }

    response = requests.get(API_endpoint, params=request_params)
    response_json = response.json()

    os.makedirs(save_file_path, exist_ok=True)

    # Save response to JSON file
    with open(f'{save_file_path}/occupancy_data_{location_slug}.json', 'w') as outfile:
        json.dump(response_json, outfile)

    return


In [4]:
location_slug = "parkplatz-graupsaege-1"
load_and_save_occupancy_data_for_location(location_slug)

## Historical occupancy data

In [30]:
def get_historical_occupancy_data_for_location(
    location_id: str,
    save_file_path: str = 'outputs'
):
    API_endpoint = f'https://data.bayerncloud.digital/api/v4/things/{location_id}/dcls_occupancy'

    request_params = {
        'token': BAYERN_CLOUD_API_KEY
    }

    response = requests.get(API_endpoint, params=request_params)
    response_json = response.json()

    # Convert response to Pandas DataFrame and preprocess it
    historical_occupancy_df = pd.DataFrame(response_json['data'], columns=['time', 'occupancy'])
    historical_occupancy_df["time"] = pd.to_datetime(historical_occupancy_df["time"])
    historical_occupancy_df.set_index("time", inplace=True)

    # Save DF as pickle file
    os.makedirs(save_file_path, exist_ok=True)
    historical_occupancy_df.to_csv(f'{save_file_path}/occupancy_data_{location_id}.csv')

    return

In [31]:
visitor_counters = {
    "tfg-lusen-1": "7453d98e-8ba7-4dfc-8054-a55c685c9e12",
    "tfg-lusen-2": "11c69a64-8600-47d9-93a8-825e655d50f5",
    "tfg-lusen-3": "0dce6a7a-e256-46b9-86cb-1189de9387c7",
    "tfg-falkenstein-1": "6d666ec4-96e3-4506-ac08-b8953c98b569",
    "tfg-falkenstein-2": "2105b493-02a8-4ae9-a6e4-bce6c866eca6",
}

In [32]:
for value in visitor_counters.values():
    print(f"Loading and saving historical occupancy data for Location ID: {value}")
    get_historical_occupancy_data_for_location(location_id=value)

Loading and saving historical occupancy data for Location ID: 7453d98e-8ba7-4dfc-8054-a55c685c9e12
Loading and saving historical occupancy data for Location ID: 11c69a64-8600-47d9-93a8-825e655d50f5
Loading and saving historical occupancy data for Location ID: 0dce6a7a-e256-46b9-86cb-1189de9387c7
Loading and saving historical occupancy data for Location ID: 6d666ec4-96e3-4506-ac08-b8953c98b569
Loading and saving historical occupancy data for Location ID: 2105b493-02a8-4ae9-a6e4-bce6c866eca6
